In [1]:
import xarray as xr
import hvplot.xarray

# Explore SM2RAIN-ASCAT dataset

In this notebook we explore the SM2RAIN-ASCAT dataset v2.1.2n from Brocca et al. [Brocca et al. 10.5281/zenodo.10376109](https://zenodo.org/records/10376109) (see corresponding [publication](https://essd.copernicus.org/articles/11/1583/2019/)). See [`download-data.bsh`](../download-data.bsh) for how to retrieve (part of) the dataset.

In [2]:
# define bounding box
lon_min = -74
lon_max = -70
lat_min = -42
lat_max = -38

In [3]:
# open and concatenate NetCDF files
ds = xr.open_mfdataset(
    "../data/SM2RAIN_ASCAT_01_v2-1-2n_year_*.nc", 
    chunks="auto", 
    engine="h5netcdf",
)

In [4]:
# extract rainfall, crop to bounding box and resample to yearly
rainfall = ds["rainfall"] \
  .sel(lon=slice(lon_min, lon_max), lat=slice(lat_max, lat_min)) \
  .resample(time="YS") \
  .sum() \
  .load()
# update  attribute
rainfall.attrs["long_name"] = "Yearly rainfall in mm"

In [5]:
q98 = rainfall.quantile(0.98)

In [6]:
# generate plot
plot = rainfall.hvplot.quadmesh(
  tiles=True, 
  alpha=0.6, 
  frame_height=600,
  frame_width=600,
  widget_location='bottom',
  clim=(0, q98),
)

plot

Column
    [0] HoloViews(DynamicMap, sizing_mode='fixed', widget_location='bottom')
    [1] WidgetBox(align=('center', 'end'))
        [0] DiscreteSlider(name='time', options={'2007-01-01 00:00:00': np...}, value=np.datetime64('2007-01-01T...)

In [7]:
# export
hvplot.save(plot, "rainfall.html", embed=True)